<a href="https://colab.research.google.com/github/bil-ardhry/fastai_course_22/blob/main/Linear_Model_and_Neural_Net_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>